In [ ]:
# function ClickConnect(){
#     console.log("Working");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }
# setInterval(ClickConnect, 30000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/"My Drive"/"Colab Notebooks"/master_project/HAN

In [ ]:
%%capture
!pip install flair

In [ ]:
import pandas as pd
import tqdm
import pickle
import sys
import time
from flair.data import Sentence
from flair.models import SequenceTagger
from timeit import default_timer as timer
sys.path.append('..')
from utilities import *

### Data

In [ ]:
df_all = pd.read_pickle('df_all.pkl')

In [ ]:
df_all

In [ ]:
# POS-tagging
tagger = SequenceTagger.load('pos')

def get_pos(sentence):
    sent_obj = Sentence(sentence)
    tagger.predict(sent_obj)
    return " ".join([ptag.tag for ptag in sent_obj.get_spans('pos')])

2020-04-24 20:28:53,098 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/POS-ontonotes--h256-l1-b32-p3-0.5-%2Bglove%2Bnews-forward%2Bnews-backward-normal-locked0.5-word0.05--v0.4_0/en-pos-ontonotes-v0.4.pt not found in cache, downloading to /tmp/tmpd7mnzxgd


100%|██████████| 432218302/432218302 [00:26<00:00, 16227892.07B/s]

2020-04-24 20:29:20,465 copying /tmp/tmpd7mnzxgd to cache at /root/.flair/models/en-pos-ontonotes-v0.4.pt


2020-04-24 20:29:21,627 removing temp file /tmp/tmpd7mnzxgd
2020-04-24 20:29:21,877 loading file /root/.flair/models/en-pos-ontonotes-v0.4.pt


In [ ]:
to_pos = df_all["words"].to_list()

In [ ]:
with open('progress_dict_old.pickle', 'rb') as handle:
    progress_dict_old = pickle.load(handle) 

In [ ]:
try:
    with open('progress_dict.pickle', 'rb') as handle:
        progress_dict = pickle.load(handle) 
    print("Loaded Existing Progress Dict")
except FileNotFoundError as e:
    print("Created New Progress Dict")
    progress_dict = {}
    with open('progress_dict.pickle', 'wb') as handle:
        pickle.dump(progress_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

while len(progress_dict) != len(set(df_all.words)):
    start = timer()
    print("Opening File...")
    with open('progress_dict.pickle', 'rb') as handle:
        progress_dict = pickle.load(handle)    
    to_pos = df_all["words"].to_list()[len(progress_dict):]
    print(f"Start taggging, left in queue: {len(to_pos)}")
    for idx, sentence in enumerate(to_pos):
        try:
            existing_pos = progress_dict_old[sentence]
            progress_dict[sentence] = existing_pos
        except KeyError as e:
            progress_dict[sentence] = get_pos(sentence)
        if idx in [5000, 10000, 15000, 20000]:
            print(f":: At step {idx}")
        if idx == 25000:
            break
    print("Saving File...")
    with open('progress_dict.pickle', 'wb') as handle:
        pickle.dump(progress_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    time.sleep(5)
    print(f"Minutes: {round((timer() - start)/60, 2)}")

Loaded Existing Progress Dict
Created New Progress Dict
Opening File...
Start taggging, left in queue: 800000
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.45
Opening File...
Start taggging, left in queue: 775008
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.46
Opening File...
Start taggging, left in queue: 750043
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.43
Opening File...
Start taggging, left in queue: 725122
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.5
Opening File...
Start taggging, left in queue: 700270
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.48
Opening File...
Start taggging, left in queue: 675496
:: At step 5000
:: At step 10000
:: At step 15000
:: At step 20000
Saving File...
Minutes: 5.48
Opening File...
Start taggging, left in queue

In [ ]:
print(len(progress_dict))

793164


In [ ]:
with open('progress_dict.pickle', 'rb') as handle:
    progress_dict = pickle.load(handle) 
pos_sentences = []
to_pos = df_all["words"].to_list()
for sent in to_pos:
    pos_sentences.append(progress_dict[sent])

In [ ]:
df_all["pos"] = pos_sentences

In [ ]:
df_all

,words,categories,is_which_set,pos
0,i figured betty crocker would make a great cake,0,train,PRON VERB ADJ NOUN AUX VERB DET ADJ NOUN
1,i was offered the opportunity to taste this pr...,0,train,PRON VERB VERB DET NOUN PART VERB DET NOUN ADP...
2,it is small and hard to depress when wearing f...,0,train,PRON VERB ADJ CCONJ ADJ PART VERB ADV VERB ADJ...
3,the grids do not remove but are easy to wipe d...,1,train,DET NOUN VERB ADV VERB CCONJ VERB ADJ PART VER...
4,i mean no secret characters and you change you...,0,train,PRON VERB DET ADJ NOUN CCONJ PRON VERB PRON NO...
...,...,...,...,...
799995,the size is just right for a meal,1,test,DET NOUN VERB ADV ADJ ADP DET NOUN
799996,clean up is a breeze due to the nice slick finish,1,test,VERB ADP VERB DET NOUN ADP ADP DET ADJ ADJ NOUN
799997,unfortunately the initial mouse arrived missin...,0,test,ADV DET ADJ NOUN VERB VERB DET NOUN PRON VERB ...
799998,we have used this in the oven microwave safe f...,1,test,PRON VERB VERB DET ADP DET ADJ NOUN ADJ NOUN C...


In [ ]:
from sklearn.utils import shuffle
df_all = shuffle(df_all)
df_all.reset_index(inplace=True, drop=True)

In [ ]:
from collections import Counter
Counter(df_all[df_all["is_which_set"]=="test"].categories)

Counter({0: 50000, 1: 50000})

In [ ]:
df_all.to_pickle('df_all.pkl')

In [ ]:
df_all = pd.read_pickle('df_all.pkl')

In [ ]:
df_all

,words,categories,is_which_set,pos
0,in bought this when in bought the pop maker,0,train,ADP VERB DET ADV ADV VERB DET NOUN NOUN
1,as for pop embellishing well that wasn to too ...,0,train,ADP ADP NOUN VERB INTJ DET VERB PART ADV ADJ P...
2,in do save the money and spend it instead on e...,0,train,ADP VERB VERB DET NOUN CCONJ VERB PRON ADV ADP...
3,did not really care many of the cakes at all,0,train,VERB ADV ADV VERB ADJ ADP DET NOUN ADV ADV
4,not up to normal standing for wilton yearbooks...,0,train,ADV ADV ADP ADJ NOUN ADP NOUN NOUN ADP DET NOUN
...,...,...,...,...
799995,as cases though it is not the best but it is c...,1,test,ADP NOUN ADP PRON VERB ADV DET ADJ CCONJ PRON ...
799996,not as lot to complain about with this case at...,1,test,ADV ADV NOUN PART VERB ADP ADP DET NOUN ADP DE...
799997,in was happy with the tpu case in bought for m...,1,test,ADP VERB ADJ ADP DET NOUN NOUN ADP VERB ADP PR...
799998,it fits the phone perfectly the cut outs are a...,1,test,PRON VERB DET NOUN ADV DET NOUN NOUN VERB DET ...
